In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [2]:
import os
# %pip install python-dotenv
from dotenv import load_dotenv

In [3]:
load_dotenv('./env/.env')

True

In [4]:
# Create a Spark session with your AWS Credentials
conf = (
    SparkConf()
    .setAppName("sparkQLwithS3") # replace with your desired name
    .set("spark.jars.packages", "io.delta:delta-core_2.12:2.3.0,org.apache.hadoop:hadoop-aws:3.3.2")
    .set("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    # .set("spark.sql.shuffle.partitions", "4") # default is 200 partitions which is too many for local
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set("com.amazonaws.services.s3.enableV4", "true")
    .set("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com")
    # .set("spark.hadoop.fs.s3a.aws.credentials.provider", "com.amazonaws.auth.profile.ProfileCredentialsProvider")
    .set("spark.hadoop.fs.s3a.access.key", os.environ['aws_access_key_id'])
    .set("spark.hadoop.fs.s3a.secret.key", os.environ['aws_secret_access_key'])
    .setMaster("spark://34.125.136.103:30077")
)


In [5]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()
df = spark.read.format('json').load('s3a://jolajoayo-spark-0001/spark2-sql/airports/airport-codes.csv.json')


:: loading settings :: url = jar:file:/root/miniconda3/envs/spark341/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0f89a2b4-dac9-400e-9f5e-646f60fd486f;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 645ms :: artifacts dl 21ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.1026 from central in [default]
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.2

Py4JError: An error occurred while calling o46.load

In [ ]:
df.show(5, truncate=False)


In [ ]:
df.printSchema()

In [ ]:
df.createOrReplaceTempView("airportcodes")
viewdf = spark.sql(\
                   "SELECT iso_region, name, type "\
                   "FROM airportcodes "\
                   "WHERE iso_country = 'US' "\
                   )

In [ ]:
viewdf.show()

In [ ]:
viewdf.write.format('csv').option('header','true').\
save('s3a://jolajoayo-spark-0001/spark2-sql/airports/airportcodes.csv',mode='overwrite')

In [6]:
spark.stop()
